# Machine Learning ML Script ML XGBoost 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#For ML

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split
import pickle
from sklearn.model_selection import RandomizedSearchCV

In [2]:
db = pd.read_csv('main_car_database_for_ML.csv')

In [3]:
db

,Unnamed: 0,Maker,Model,Description,Name,Year,City,Color,Power_cv,Motor,...,Doors,Seats,Cylinders,Cylindernumber,Maxvel,Urbancons,Highwaycons,Meancons,Emissions,Segment
0,0,NISSAN,Micra,IG-T S&S Acenta 90,NISSAN Micra IG-T S&S Acenta 90,2018,Las Palmas,azul,90,Manual,...,5,5,0.898,3,175.000000,6.3,4.5,5.2,118,SegmentB
1,1,RENAULT,Clio,1.5dCi eco2 S&S Energy Business 90,RENAULT Clio 1.5dCi eco2 S&S Energy Business 90,2016,Granada,blanco,90,Manual,...,5,5,1.461,4,178.000000,4.1,3.3,3.6,93,SegmentC
2,2,KIA,Cee´d,1.4CRDI WGT Business 90,KIA Cee´d 1.4CRDI WGT Business 90,2017,Sevilla,negro,90,Manual,...,5,5,1.396,4,173.000000,5.1,3.7,4.2,109,SegmentC
3,3,RENAULT,Clio,1.5dCi eco2 Energy Authentique 75,RENAULT Clio 1.5dCi eco2 Energy Authentique 75,2014,Barcelona,plata,75,Manual,...,5,5,1.461,4,168.000000,4.3,3.2,3.6,85,SegmentC
4,4,OPEL,Corsa,1.4 Selective 90,OPEL Corsa 1.4 Selective 90,2018,Málaga,gris,90,Manual,...,5,5,1.398,4,175.000000,6.5,4.3,5.1,120,SegmentC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3423,3436,FORD,Focus,1.0 Ecoboost Auto-S&S ST-Line 125,FORD Focus 1.0 Ecoboost Auto-S&S ST-Line 125,2017,Albacete,azul,125,Manual,...,5,5,0.999,3,181.506462,5.7,4.1,4.7,108,SegmentC
3424,3437,RENAULT,Mégane,1.2 TCe Energy Tech Road 97kW,RENAULT Mégane 1.2 TCe Energy Tech Road 97kW,2017,Madrid,Other,130,Manual,...,5,5,1.197,4,197.000000,6.8,4.5,5.3,119,SegmentC
3425,3438,FORD,Fiesta,1.0 EcoBoost S/S Titanium 100,FORD Fiesta 1.0 EcoBoost S/S Titanium 100,2017,Madrid,azul,100,Manual,...,5,5,0.998,3,183.000000,5.4,3.6,4.3,110,SegmentC
3426,3439,RENAULT,Clio,Sport Tourer 1.5dCi Energy Business 55kW,RENAULT Clio Sport Tourer 1.5dCi Energy Busine...,2019,Madrid,blanco,75,Manual,...,5,5,1.461,4,168.000000,4.3,3.2,3.6,106,SegmentC


In [4]:
db.drop(['Unnamed: 0'], axis=1,inplace=True)

In [5]:
#Getting Horse Power from CV (1 cv to hp = 0.98592 hp)
db['HP'] = db['Power_cv'] * .98592

In [6]:
db.columns

Index(['Maker', 'Model', 'Description', 'Name', 'Year', 'City', 'Color',
       'Power_cv', 'Motor', 'Chasis', 'Kilometers', 'Fuel', 'Price_euro',
       'Link', 'Length_mm', 'Width_mm', 'Height_mm', 'Wheelbase_mm',
       'Weight_kg', 'Doors', 'Seats', 'Cylinders', 'Cylindernumber', 'Maxvel',
       'Urbancons', 'Highwaycons', 'Meancons', 'Emissions', 'Segment', 'HP'],
      dtype='object')

In [86]:
len(db['Maker'].unique())

34

Year classification by Kilomenters

In [7]:
db['Year'].unique()

array([2018, 2016, 2017, 2014, 2015, 2019, 2021, 2020, 2022], dtype=int64)

In [8]:
def class_segment(dataframe):
    conditions = [(dataframe['Year'] == 2014) & (dataframe['Kilometers'] <= 135000 ),
                  (dataframe['Year'] == 2014) & (dataframe['Kilometers'] > 135000 )  & (dataframe['Kilometers'] <=243000),
                  (dataframe['Year'] == 2014) & (dataframe['Kilometers'] > 243000),
                  
                  (dataframe['Year'] == 2015) & (dataframe['Kilometers'] <= 12000 ),
                  (dataframe['Year'] == 2015) & (dataframe['Kilometers'] > 12000 )  & (dataframe['Kilometers'] <=216000),
                  (dataframe['Year'] == 2015) & (dataframe['Kilometers'] > 216000),
                  
                  (dataframe['Year'] == 2016) & (dataframe['Kilometers'] <= 105000),
                  (dataframe['Year'] == 2016) & (dataframe['Kilometers'] > 105000 )  & (dataframe['Kilometers'] <=189000),
                  (dataframe['Year'] == 2016) & (dataframe['Kilometers'] > 189000),
                  
                  (dataframe['Year'] == 2017) & (dataframe['Kilometers'] <= 90000 ),
                  (dataframe['Year'] == 2017) & (dataframe['Kilometers'] > 90000 )  & (dataframe['Kilometers'] <=162000),
                  (dataframe['Year'] == 2017) & (dataframe['Kilometers'] > 162000),
                  
                  (dataframe['Year'] == 2018) & (dataframe['Kilometers'] <= 75000 ),
                  (dataframe['Year'] == 2018) & (dataframe['Kilometers'] > 75000 )  & (dataframe['Kilometers'] <=135000),
                  (dataframe['Year'] == 2018) & (dataframe['Kilometers'] > 135000),
                  
                  (dataframe['Year'] == 2019) & (dataframe['Kilometers'] <= 60000 ),
                  (dataframe['Year'] == 2019) & (dataframe['Kilometers'] > 60000 )  & (dataframe['Kilometers'] <=108000),
                  (dataframe['Year'] == 2019) & (dataframe['Kilometers'] > 108000),
                  
                  (dataframe['Year'] == 2020) & (dataframe['Kilometers'] <= 45000 ),
                  (dataframe['Year'] == 2020) & (dataframe['Kilometers'] > 45000 )  & (dataframe['Kilometers'] <=81000),
                  (dataframe['Year'] == 2020) & (dataframe['Kilometers'] > 81000),
                  
                  (dataframe['Year'] == 2021) & (dataframe['Kilometers'] <= 30000 ),
                  (dataframe['Year'] == 2021) & (dataframe['Kilometers'] > 30000 )  & (dataframe['Kilometers'] <=54000),
                  (dataframe['Year'] == 2021) & (dataframe['Kilometers'] > 54000),
                  
                  (dataframe['Year'] == 2022) & (dataframe['Kilometers'] <=15000 ),
                  (dataframe['Year'] == 2022) & (dataframe['Kilometers'] > 15000 )  & (dataframe['Kilometers'] <=27000),
                  (dataframe['Year'] == 2022) & (dataframe['Kilometers'] > 27000),
                ]                
                  

    choices =['Low','Mid','High','Low','Mid','High','Low','Mid','High','Low','Mid','High','Low','Mid','High','Low','Mid','High',
             'Low','Mid','High','Low','Mid','High','Low','Mid','High']

    dataframe['Kilometers_Class'] = np.select(conditions,choices)

    return dataframe

In [9]:
class_segment(db)

,Maker,Model,Description,Name,Year,City,Color,Power_cv,Motor,Chasis,...,Cylinders,Cylindernumber,Maxvel,Urbancons,Highwaycons,Meancons,Emissions,Segment,HP,Kilometers_Class
0,NISSAN,Micra,IG-T S&S Acenta 90,NISSAN Micra IG-T S&S Acenta 90,2018,Las Palmas,azul,90,Manual,Berlina,...,0.898,3,175.000000,6.3,4.5,5.2,118,SegmentB,88.7328,Low
1,RENAULT,Clio,1.5dCi eco2 S&S Energy Business 90,RENAULT Clio 1.5dCi eco2 S&S Energy Business 90,2016,Granada,blanco,90,Manual,Berlina,...,1.461,4,178.000000,4.1,3.3,3.6,93,SegmentC,88.7328,Mid
2,KIA,Cee´d,1.4CRDI WGT Business 90,KIA Cee´d 1.4CRDI WGT Business 90,2017,Sevilla,negro,90,Manual,Berlina,...,1.396,4,173.000000,5.1,3.7,4.2,109,SegmentC,88.7328,Low
3,RENAULT,Clio,1.5dCi eco2 Energy Authentique 75,RENAULT Clio 1.5dCi eco2 Energy Authentique 75,2014,Barcelona,plata,75,Manual,Berlina,...,1.461,4,168.000000,4.3,3.2,3.6,85,SegmentC,73.9440,Low
4,OPEL,Corsa,1.4 Selective 90,OPEL Corsa 1.4 Selective 90,2018,Málaga,gris,90,Manual,Berlina,...,1.398,4,175.000000,6.5,4.3,5.1,120,SegmentC,88.7328,Mid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3423,FORD,Focus,1.0 Ecoboost Auto-S&S ST-Line 125,FORD Focus 1.0 Ecoboost Auto-S&S ST-Line 125,2017,Albacete,azul,125,Manual,Berlina,...,0.999,3,181.506462,5.7,4.1,4.7,108,SegmentC,123.2400,Low
3424,RENAULT,Mégane,1.2 TCe Energy Tech Road 97kW,RENAULT Mégane 1.2 TCe Energy Tech Road 97kW,2017,Madrid,Other,130,Manual,Berlina,...,1.197,4,197.000000,6.8,4.5,5.3,119,SegmentC,128.1696,Low
3425,FORD,Fiesta,1.0 EcoBoost S/S Titanium 100,FORD Fiesta 1.0 EcoBoost S/S Titanium 100,2017,Madrid,azul,100,Manual,Berlina,...,0.998,3,183.000000,5.4,3.6,4.3,110,SegmentC,98.5920,Low
3426,RENAULT,Clio,Sport Tourer 1.5dCi Energy Business 55kW,RENAULT Clio Sport Tourer 1.5dCi Energy Busine...,2019,Madrid,blanco,75,Manual,Stationwagon,...,1.461,4,168.000000,4.3,3.2,3.6,106,SegmentC,73.9440,High


In [10]:
db['Kilometers_Class'].unique()

array(['Low', 'Mid', 'High'], dtype=object)

In [11]:
db['Kilometers_Class'].value_counts()

Low     1779
Mid     1441
High     208
Name: Kilometers_Class, dtype: int64

In [12]:
db_transformed =pd.get_dummies(db, columns =['Kilometers_Class','Motor','Segment','Maker','Fuel'])
db_transformed

,Model,Description,Name,Year,City,Color,Power_cv,Chasis,Kilometers,Price_euro,...,Maker_SKODA,Maker_SMART,Maker_SSANGYONG,Maker_SUBARU,Maker_SUZUKI,Maker_TOYOTA,Maker_VOLKSWAGEN,Maker_VOLVO,Fuel_Diesel,Fuel_Gasoline
0,Micra,IG-T S&S Acenta 90,NISSAN Micra IG-T S&S Acenta 90,2018,Las Palmas,azul,90,Berlina,51773,13450,...,0,0,0,0,0,0,0,0,0,1
1,Clio,1.5dCi eco2 S&S Energy Business 90,RENAULT Clio 1.5dCi eco2 S&S Energy Business 90,2016,Granada,blanco,90,Berlina,115000,10500,...,0,0,0,0,0,0,0,0,1,0
2,Cee´d,1.4CRDI WGT Business 90,KIA Cee´d 1.4CRDI WGT Business 90,2017,Sevilla,negro,90,Berlina,71565,12290,...,0,0,0,0,0,0,0,0,1,0
3,Clio,1.5dCi eco2 Energy Authentique 75,RENAULT Clio 1.5dCi eco2 Energy Authentique 75,2014,Barcelona,plata,75,Berlina,109000,8600,...,0,0,0,0,0,0,0,0,1,0
4,Corsa,1.4 Selective 90,OPEL Corsa 1.4 Selective 90,2018,Málaga,gris,90,Berlina,84340,11300,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3423,Focus,1.0 Ecoboost Auto-S&S ST-Line 125,FORD Focus 1.0 Ecoboost Auto-S&S ST-Line 125,2017,Albacete,azul,125,Berlina,46188,15700,...,0,0,0,0,0,0,0,0,0,1
3424,Mégane,1.2 TCe Energy Tech Road 97kW,RENAULT Mégane 1.2 TCe Energy Tech Road 97kW,2017,Madrid,Other,130,Berlina,55400,14765,...,0,0,0,0,0,0,0,0,0,1
3425,Fiesta,1.0 EcoBoost S/S Titanium 100,FORD Fiesta 1.0 EcoBoost S/S Titanium 100,2017,Madrid,azul,100,Berlina,40480,12899,...,0,0,0,0,0,0,0,0,0,1
3426,Clio,Sport Tourer 1.5dCi Energy Business 55kW,RENAULT Clio Sport Tourer 1.5dCi Energy Busine...,2019,Madrid,blanco,75,Stationwagon,114000,11500,...,0,0,0,0,0,0,0,0,1,0


In [13]:
len(db['Maxvel'].unique())

80

In [14]:
print('Min: ',db['Maxvel'].min())
print('Avg: ',db['Maxvel'].mean())
print('High: ',db['Maxvel'].max())

Min:  140.0
Avg:  181.47746111427315
High:  240.0


In [15]:
db_transformed.columns

Index(['Model', 'Description', 'Name', 'Year', 'City', 'Color', 'Power_cv',
       'Chasis', 'Kilometers', 'Price_euro', 'Link', 'Length_mm', 'Width_mm',
       'Height_mm', 'Wheelbase_mm', 'Weight_kg', 'Doors', 'Seats', 'Cylinders',
       'Cylindernumber', 'Maxvel', 'Urbancons', 'Highwaycons', 'Meancons',
       'Emissions', 'HP', 'Kilometers_Class_High', 'Kilometers_Class_Low',
       'Kilometers_Class_Mid', 'Motor_Automatic', 'Motor_Manual',
       'Segment_SegmentA', 'Segment_SegmentB', 'Segment_SegmentC',
       'Segment_SegmentD', 'Segment_SegmentE', 'Maker_ABARTH', 'Maker_ALFA',
       'Maker_AUDI', 'Maker_BMW', 'Maker_CHEVROLET', 'Maker_CITROEN',
       'Maker_DACIA', 'Maker_DS', 'Maker_FIAT', 'Maker_FORD', 'Maker_HONDA',
       'Maker_HYUNDAI', 'Maker_INFINITI', 'Maker_JAGUAR', 'Maker_JEEP',
       'Maker_KIA', 'Maker_LEXUS', 'Maker_MAZDA', 'Maker_MERCEDES-BENZ',
       'Maker_MINI', 'Maker_MITSUBISHI', 'Maker_NISSAN', 'Maker_OPEL',
       'Maker_PEUGEOT', 'Maker_RENAULT', 'M

In [16]:
db_ml = db_transformed[['Year', 'HP','Maxvel','Kilometers', 'Kilometers_Class_High',
       'Kilometers_Class_Low', 'Kilometers_Class_Mid', 'Motor_Automatic',
       'Motor_Manual', 'Segment_SegmentA', 'Segment_SegmentB',
       'Segment_SegmentC', 'Segment_SegmentD', 'Segment_SegmentE',
       'Maker_ABARTH', 'Maker_ALFA', 'Maker_AUDI', 'Maker_BMW',
       'Maker_CHEVROLET', 'Maker_CITROEN', 'Maker_DACIA', 'Maker_DS',
       'Maker_FIAT', 'Maker_FORD', 'Maker_HONDA', 'Maker_HYUNDAI',
       'Maker_INFINITI', 'Maker_JAGUAR', 'Maker_JEEP', 'Maker_KIA',
       'Maker_LEXUS', 'Maker_MAZDA', 'Maker_MERCEDES-BENZ', 'Maker_MINI',
       'Maker_MITSUBISHI', 'Maker_NISSAN', 'Maker_OPEL', 'Maker_PEUGEOT',
       'Maker_RENAULT', 'Maker_SEAT', 'Maker_SKODA', 'Maker_SMART',
       'Maker_SSANGYONG', 'Maker_SUBARU', 'Maker_SUZUKI', 'Maker_TOYOTA',
       'Maker_VOLKSWAGEN', 'Maker_VOLVO', 'Fuel_Diesel',
       'Fuel_Gasoline','Price_euro','Cylindernumber']]

In [17]:
db_ml

,Year,HP,Maxvel,Kilometers,Kilometers_Class_High,Kilometers_Class_Low,Kilometers_Class_Mid,Motor_Automatic,Motor_Manual,Segment_SegmentA,...,Maker_SSANGYONG,Maker_SUBARU,Maker_SUZUKI,Maker_TOYOTA,Maker_VOLKSWAGEN,Maker_VOLVO,Fuel_Diesel,Fuel_Gasoline,Price_euro,Cylindernumber
0,2018,88.7328,175.000000,51773,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,13450,3
1,2016,88.7328,178.000000,115000,0,0,1,0,1,0,...,0,0,0,0,0,0,1,0,10500,4
2,2017,88.7328,173.000000,71565,0,1,0,0,1,0,...,0,0,0,0,0,0,1,0,12290,4
3,2014,73.9440,168.000000,109000,0,1,0,0,1,0,...,0,0,0,0,0,0,1,0,8600,4
4,2018,88.7328,175.000000,84340,0,0,1,0,1,0,...,0,0,0,0,0,0,0,1,11300,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3423,2017,123.2400,181.506462,46188,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,15700,3
3424,2017,128.1696,197.000000,55400,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,14765,4
3425,2017,98.5920,183.000000,40480,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,12899,3
3426,2019,73.9440,168.000000,114000,1,0,0,0,1,0,...,0,0,0,0,0,0,1,0,11500,4


In [18]:
X = db_ml[['Year', 'HP','Maxvel', 'Kilometers_Class_High',
       'Kilometers_Class_Low', 'Kilometers_Class_Mid', 'Motor_Automatic',
       'Motor_Manual', 'Segment_SegmentA', 'Segment_SegmentB',
       'Segment_SegmentC', 'Segment_SegmentD', 'Segment_SegmentE',
       'Maker_ABARTH', 'Maker_ALFA', 'Maker_AUDI', 'Maker_BMW',
       'Maker_CHEVROLET', 'Maker_CITROEN', 'Maker_DACIA', 'Maker_DS',
       'Maker_FIAT', 'Maker_FORD', 'Maker_HONDA', 'Maker_HYUNDAI',
       'Maker_INFINITI', 'Maker_JAGUAR', 'Maker_JEEP', 'Maker_KIA',
       'Maker_LEXUS', 'Maker_MAZDA', 'Maker_MERCEDES-BENZ', 'Maker_MINI',
       'Maker_MITSUBISHI', 'Maker_NISSAN', 'Maker_OPEL', 'Maker_PEUGEOT',
       'Maker_RENAULT', 'Maker_SEAT', 'Maker_SKODA', 'Maker_SMART',
       'Maker_SSANGYONG', 'Maker_SUBARU', 'Maker_SUZUKI', 'Maker_TOYOTA',
       'Maker_VOLKSWAGEN', 'Maker_VOLVO', 'Fuel_Diesel',
       'Fuel_Gasoline']]

In [19]:
X

,Year,HP,Maxvel,Kilometers_Class_High,Kilometers_Class_Low,Kilometers_Class_Mid,Motor_Automatic,Motor_Manual,Segment_SegmentA,Segment_SegmentB,...,Maker_SKODA,Maker_SMART,Maker_SSANGYONG,Maker_SUBARU,Maker_SUZUKI,Maker_TOYOTA,Maker_VOLKSWAGEN,Maker_VOLVO,Fuel_Diesel,Fuel_Gasoline
0,2018,88.7328,175.000000,0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,1
1,2016,88.7328,178.000000,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2017,88.7328,173.000000,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,2014,73.9440,168.000000,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
4,2018,88.7328,175.000000,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3423,2017,123.2400,181.506462,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
3424,2017,128.1696,197.000000,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
3425,2017,98.5920,183.000000,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
3426,2019,73.9440,168.000000,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [20]:
y  =db_ml['Price_euro']

In [21]:
y

0       13450
1       10500
2       12290
3        8600
4       11300
        ...  
3423    15700
3424    14765
3425    12899
3426    11500
3427    14499
Name: Price_euro, Length: 3428, dtype: int64

In [22]:
#Split in training and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

Normalization

In [23]:
#let's apply a normalization for the features
from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import StandardScaler

# create normalization object from scikit learn package, and "fit" it to the features in hand
normalizer = MinMaxScaler()
# notice how we only use the **X_train** data to fit?
# we want to use only the training data to normalize (establishing maximum and minimum values) to avoid data "leakage" from the test dataset
# if we used data from the test dataset, the test results would be biased by having some info from the test set
normalizer = normalizer.fit(X_train)

In [24]:
X_train_normalized = normalizer.transform(X_train) 
X_train_normalized = pd.DataFrame(X_train_normalized,columns=X_train.columns)
X_train_normalized

,Year,HP,Maxvel,Kilometers_Class_High,Kilometers_Class_Low,Kilometers_Class_Mid,Motor_Automatic,Motor_Manual,Segment_SegmentA,Segment_SegmentB,...,Maker_SKODA,Maker_SMART,Maker_SSANGYONG,Maker_SUBARU,Maker_SUZUKI,Maker_TOYOTA,Maker_VOLKSWAGEN,Maker_VOLVO,Fuel_Diesel,Fuel_Gasoline
0,0.625,0.062500,0.210526,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.375,0.486111,0.600000,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.375,0.062500,0.252632,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.125,0.041667,0.189474,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.500,0.173611,0.315789,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2566,0.125,0.062500,0.210526,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2567,0.500,0.416667,0.589474,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2568,0.000,0.104167,0.242105,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2569,0.250,0.270833,0.421053,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [25]:
X_test_normalized = normalizer.transform(X_test)
X_test_normalized = pd.DataFrame(X_test_normalized,columns=X_test.columns)
X_test_normalized

,Year,HP,Maxvel,Kilometers_Class_High,Kilometers_Class_Low,Kilometers_Class_Mid,Motor_Automatic,Motor_Manual,Segment_SegmentA,Segment_SegmentB,...,Maker_SKODA,Maker_SMART,Maker_SSANGYONG,Maker_SUBARU,Maker_SUZUKI,Maker_TOYOTA,Maker_VOLKSWAGEN,Maker_VOLVO,Fuel_Diesel,Fuel_Gasoline
0,0.375,0.062500,0.210526,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.125,0.388889,0.631579,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.250,0.527778,0.600000,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.125,0.416667,0.436910,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.375,0.277778,0.484211,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
852,0.625,0.243056,0.436910,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
853,0.250,0.555556,0.684211,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
854,0.125,0.381944,0.347368,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
855,0.875,0.152778,0.294737,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


# XGBoost

In [108]:
import xgboost

In [109]:
xg_reg = xgboost.XGBRegressor()

In [110]:
len(X_train)
len(y_train)

2571

In [111]:
model = xg_reg.fit(X_train,y_train)
model.score(X_test, y_test)


0.6821045830184922

In [112]:
model1 = xg_reg.fit(X_train_normalized,y_train)
model1.score(X_test_normalized, y_test)

-0.18821859370921912

# The model improved from linear regresion 

Feature engineering, adjusting X to get better score

In [31]:
#Bining Max Vel

In [32]:
db_transformed['Maxvel']

0       175.000000
1       178.000000
2       173.000000
3       168.000000
4       175.000000
           ...    
3423    181.506462
3424    197.000000
3425    183.000000
3426    168.000000
3427    181.506462
Name: Maxvel, Length: 3428, dtype: float64

In [33]:
print('Min: ',db['Maxvel'].min())
print('Avg: ',db['Maxvel'].mean())
print('High: ',db['Maxvel'].max())

Min:  140.0
Avg:  181.47746111427315
High:  240.0


In [34]:
db_ml['Maxvel_class'] = pd.cut(x=db_ml['Maxvel'], bins=[140, 170, 200, 240], labels=['slow', 'avg', 'fast'])

C:\Users\SERGIO~1\AppData\Local\Temp/ipykernel_13808/1633462753.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_ml['Maxvel_class'] = pd.cut(x=db_ml['Maxvel'], bins=[140, 170, 200, 240], labels=['slow', 'avg', 'fast'])


In [35]:
db_ml =pd.get_dummies(db_ml, columns =['Maxvel_class'])

In [36]:
db_ml

,Year,HP,Maxvel,Kilometers,Kilometers_Class_High,Kilometers_Class_Low,Kilometers_Class_Mid,Motor_Automatic,Motor_Manual,Segment_SegmentA,...,Maker_TOYOTA,Maker_VOLKSWAGEN,Maker_VOLVO,Fuel_Diesel,Fuel_Gasoline,Price_euro,Cylindernumber,Maxvel_class_slow,Maxvel_class_avg,Maxvel_class_fast
0,2018,88.7328,175.000000,51773,0,1,0,0,1,0,...,0,0,0,0,1,13450,3,0,1,0
1,2016,88.7328,178.000000,115000,0,0,1,0,1,0,...,0,0,0,1,0,10500,4,0,1,0
2,2017,88.7328,173.000000,71565,0,1,0,0,1,0,...,0,0,0,1,0,12290,4,0,1,0
3,2014,73.9440,168.000000,109000,0,1,0,0,1,0,...,0,0,0,1,0,8600,4,1,0,0
4,2018,88.7328,175.000000,84340,0,0,1,0,1,0,...,0,0,0,0,1,11300,4,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3423,2017,123.2400,181.506462,46188,0,1,0,0,1,0,...,0,0,0,0,1,15700,3,0,1,0
3424,2017,128.1696,197.000000,55400,0,1,0,0,1,0,...,0,0,0,0,1,14765,4,0,1,0
3425,2017,98.5920,183.000000,40480,0,1,0,0,1,0,...,0,0,0,0,1,12899,3,0,1,0
3426,2019,73.9440,168.000000,114000,1,0,0,0,1,0,...,0,0,0,1,0,11500,4,1,0,0


In [37]:
X = db_ml[['Year', 'HP', 'Kilometers_Class_High',
       'Kilometers_Class_Low', 'Kilometers_Class_Mid', 'Motor_Automatic',
       'Motor_Manual', 'Segment_SegmentA', 'Segment_SegmentB',
       'Segment_SegmentC', 'Segment_SegmentD', 'Segment_SegmentE',
       'Maker_ABARTH', 'Maker_ALFA', 'Maker_AUDI', 'Maker_BMW',
       'Maker_CHEVROLET', 'Maker_CITROEN', 'Maker_DACIA', 'Maker_DS',
       'Maker_FIAT', 'Maker_FORD', 'Maker_HONDA', 'Maker_HYUNDAI',
       'Maker_INFINITI', 'Maker_JAGUAR', 'Maker_JEEP', 'Maker_KIA',
       'Maker_LEXUS', 'Maker_MAZDA', 'Maker_MERCEDES-BENZ', 'Maker_MINI',
       'Maker_MITSUBISHI', 'Maker_NISSAN', 'Maker_OPEL', 'Maker_PEUGEOT',
       'Maker_RENAULT', 'Maker_SEAT', 'Maker_SKODA', 'Maker_SMART',
       'Maker_SSANGYONG', 'Maker_SUBARU', 'Maker_SUZUKI', 'Maker_TOYOTA',
       'Maker_VOLKSWAGEN', 'Maker_VOLVO', 'Fuel_Diesel',
       'Fuel_Gasoline','Maxvel_class_slow','Maxvel_class_avg','Maxvel_class_fast']]

In [38]:
y  =db_ml['Price_euro']

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [40]:
xg_reg = xgboost.XGBRegressor()
model = xg_reg.fit(X_train,y_train)
model.score(X_test, y_test)


0.6244053814637873

In [41]:
print('Min: ',db_ml['HP'].min())
print('Avg: ',db_ml['HP'].mean())
print('High: ',db_ml['HP'].max())
print('mid: ',db_ml['HP'].max()-db_ml['HP'].min())
print('dif: ',(db_ml['HP'].max()-db_ml['HP'].min()) - db_ml['HP'].min())

Min:  59.1552
Avg:  102.0378306651109
High:  201.12768
mid:  141.97248
dif:  82.81727999999998


In [42]:
db_ml['HP_class'] = pd.cut(x=db_ml['HP'], bins=[59, 106, 153, 202], labels=['low', 'mid', 'high'])

In [43]:
db_ml

,Year,HP,Maxvel,Kilometers,Kilometers_Class_High,Kilometers_Class_Low,Kilometers_Class_Mid,Motor_Automatic,Motor_Manual,Segment_SegmentA,...,Maker_VOLKSWAGEN,Maker_VOLVO,Fuel_Diesel,Fuel_Gasoline,Price_euro,Cylindernumber,Maxvel_class_slow,Maxvel_class_avg,Maxvel_class_fast,HP_class
0,2018,88.7328,175.000000,51773,0,1,0,0,1,0,...,0,0,0,1,13450,3,0,1,0,low
1,2016,88.7328,178.000000,115000,0,0,1,0,1,0,...,0,0,1,0,10500,4,0,1,0,low
2,2017,88.7328,173.000000,71565,0,1,0,0,1,0,...,0,0,1,0,12290,4,0,1,0,low
3,2014,73.9440,168.000000,109000,0,1,0,0,1,0,...,0,0,1,0,8600,4,1,0,0,low
4,2018,88.7328,175.000000,84340,0,0,1,0,1,0,...,0,0,0,1,11300,4,0,1,0,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3423,2017,123.2400,181.506462,46188,0,1,0,0,1,0,...,0,0,0,1,15700,3,0,1,0,mid
3424,2017,128.1696,197.000000,55400,0,1,0,0,1,0,...,0,0,0,1,14765,4,0,1,0,mid
3425,2017,98.5920,183.000000,40480,0,1,0,0,1,0,...,0,0,0,1,12899,3,0,1,0,low
3426,2019,73.9440,168.000000,114000,1,0,0,0,1,0,...,0,0,1,0,11500,4,1,0,0,low


In [44]:
db_ml.columns.tolist().index('Maker_VOLKSWAGEN')

46

In [45]:
db_ml =pd.get_dummies(db_ml, columns =['HP_class'])

In [46]:
db_ml.columns

Index(['Year', 'HP', 'Maxvel', 'Kilometers', 'Kilometers_Class_High',
       'Kilometers_Class_Low', 'Kilometers_Class_Mid', 'Motor_Automatic',
       'Motor_Manual', 'Segment_SegmentA', 'Segment_SegmentB',
       'Segment_SegmentC', 'Segment_SegmentD', 'Segment_SegmentE',
       'Maker_ABARTH', 'Maker_ALFA', 'Maker_AUDI', 'Maker_BMW',
       'Maker_CHEVROLET', 'Maker_CITROEN', 'Maker_DACIA', 'Maker_DS',
       'Maker_FIAT', 'Maker_FORD', 'Maker_HONDA', 'Maker_HYUNDAI',
       'Maker_INFINITI', 'Maker_JAGUAR', 'Maker_JEEP', 'Maker_KIA',
       'Maker_LEXUS', 'Maker_MAZDA', 'Maker_MERCEDES-BENZ', 'Maker_MINI',
       'Maker_MITSUBISHI', 'Maker_NISSAN', 'Maker_OPEL', 'Maker_PEUGEOT',
       'Maker_RENAULT', 'Maker_SEAT', 'Maker_SKODA', 'Maker_SMART',
       'Maker_SSANGYONG', 'Maker_SUBARU', 'Maker_SUZUKI', 'Maker_TOYOTA',
       'Maker_VOLKSWAGEN', 'Maker_VOLVO', 'Fuel_Diesel', 'Fuel_Gasoline',
       'Price_euro', 'Cylindernumber', 'Maxvel_class_slow', 'Maxvel_class_avg',
       'Maxve

In [47]:
#Max vel with bins
X = db_ml[['Year', 'Kilometers_Class_High',
       'Kilometers_Class_Low', 'Kilometers_Class_Mid', 'Motor_Automatic',
       'Motor_Manual', 'Segment_SegmentA', 'Segment_SegmentB',
       'Segment_SegmentC', 'Segment_SegmentD', 'Segment_SegmentE',
       'Maker_ABARTH', 'Maker_ALFA', 'Maker_AUDI', 'Maker_BMW',
       'Maker_CHEVROLET', 'Maker_CITROEN', 'Maker_DACIA', 'Maker_DS',
       'Maker_FIAT', 'Maker_FORD', 'Maker_HONDA', 'Maker_HYUNDAI',
       'Maker_INFINITI', 'Maker_JAGUAR', 'Maker_JEEP', 'Maker_KIA',
       'Maker_LEXUS', 'Maker_MAZDA', 'Maker_MERCEDES-BENZ', 'Maker_MINI',
       'Maker_MITSUBISHI', 'Maker_NISSAN', 'Maker_OPEL', 'Maker_PEUGEOT',
       'Maker_RENAULT', 'Maker_SEAT', 'Maker_SKODA', 'Maker_SMART',
       'Maker_SSANGYONG', 'Maker_SUBARU', 'Maker_SUZUKI', 'Maker_TOYOTA',
       'Maker_VOLKSWAGEN', 'Maker_VOLVO', 'Fuel_Diesel',
       'Fuel_Gasoline','Maxvel_class_slow','Maxvel_class_avg','Maxvel_class_fast',
           'HP_class_low', 'HP_class_mid', 'HP_class_high']]

In [48]:
y  =db_ml['Price_euro']

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [50]:
xg_reg = xgboost.XGBRegressor()
model = xg_reg.fit(X_train,y_train)
model.score(X_test, y_test)


0.5890152526375201

In [53]:
#With out segment
X = db_ml[['Year', 'Kilometers_Class_High',
       'Kilometers_Class_Low', 'Kilometers_Class_Mid', 'Motor_Automatic',
       'Motor_Manual', 'Maker_ABARTH', 'Maker_ALFA', 'Maker_AUDI', 'Maker_BMW',
       'Maker_CHEVROLET', 'Maker_CITROEN', 'Maker_DACIA', 'Maker_DS',
       'Maker_FIAT', 'Maker_FORD', 'Maker_HONDA', 'Maker_HYUNDAI',
       'Maker_INFINITI', 'Maker_JAGUAR', 'Maker_JEEP', 'Maker_KIA',
       'Maker_LEXUS', 'Maker_MAZDA', 'Maker_MERCEDES-BENZ', 'Maker_MINI',
       'Maker_MITSUBISHI', 'Maker_NISSAN', 'Maker_OPEL', 'Maker_PEUGEOT',
       'Maker_RENAULT', 'Maker_SEAT', 'Maker_SKODA', 'Maker_SMART',
       'Maker_SSANGYONG', 'Maker_SUBARU', 'Maker_SUZUKI', 'Maker_TOYOTA',
       'Maker_VOLKSWAGEN', 'Maker_VOLVO', 'Fuel_Diesel',
       'Fuel_Gasoline','Maxvel_class_slow','Maxvel_class_avg','Maxvel_class_fast',
           'HP_class_low', 'HP_class_mid', 'HP_class_high']]
y  =db_ml['Price_euro']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
xg_reg = xgboost.XGBRegressor()
model = xg_reg.fit(X_train,y_train)
model.score(X_test, y_test)

0.49698993547890735

In [54]:
db_ml['Year'].unique()

array([2018, 2016, 2017, 2014, 2015, 2019, 2021, 2020, 2022], dtype=int64)

In [55]:
db_ml['Year_Class'] = pd.cut(x=db_ml['Year'], bins=[2013, 2016, 2019, 2022], labels=['6_9anios', '3_6anios', 'less_3anios'])

In [56]:
db_ml =pd.get_dummies(db_ml, columns =['Year_Class'])

In [57]:
db_ml.columns

Index(['Year', 'HP', 'Maxvel', 'Kilometers', 'Kilometers_Class_High',
       'Kilometers_Class_Low', 'Kilometers_Class_Mid', 'Motor_Automatic',
       'Motor_Manual', 'Segment_SegmentA', 'Segment_SegmentB',
       'Segment_SegmentC', 'Segment_SegmentD', 'Segment_SegmentE',
       'Maker_ABARTH', 'Maker_ALFA', 'Maker_AUDI', 'Maker_BMW',
       'Maker_CHEVROLET', 'Maker_CITROEN', 'Maker_DACIA', 'Maker_DS',
       'Maker_FIAT', 'Maker_FORD', 'Maker_HONDA', 'Maker_HYUNDAI',
       'Maker_INFINITI', 'Maker_JAGUAR', 'Maker_JEEP', 'Maker_KIA',
       'Maker_LEXUS', 'Maker_MAZDA', 'Maker_MERCEDES-BENZ', 'Maker_MINI',
       'Maker_MITSUBISHI', 'Maker_NISSAN', 'Maker_OPEL', 'Maker_PEUGEOT',
       'Maker_RENAULT', 'Maker_SEAT', 'Maker_SKODA', 'Maker_SMART',
       'Maker_SSANGYONG', 'Maker_SUBARU', 'Maker_SUZUKI', 'Maker_TOYOTA',
       'Maker_VOLKSWAGEN', 'Maker_VOLVO', 'Fuel_Diesel', 'Fuel_Gasoline',
       'Price_euro', 'Cylindernumber', 'Maxvel_class_slow', 'Maxvel_class_avg',
       'Maxve

In [58]:
#All variables bins and dumified in algoritm
X = db_ml[['Kilometers_Class_High',
       'Kilometers_Class_Low', 'Kilometers_Class_Mid', 'Motor_Automatic',
       'Motor_Manual', 'Segment_SegmentA', 'Segment_SegmentB',
       'Segment_SegmentC', 'Segment_SegmentD', 'Segment_SegmentE',
       'Maker_ABARTH', 'Maker_ALFA', 'Maker_AUDI', 'Maker_BMW',
       'Maker_CHEVROLET', 'Maker_CITROEN', 'Maker_DACIA', 'Maker_DS',
       'Maker_FIAT', 'Maker_FORD', 'Maker_HONDA', 'Maker_HYUNDAI',
       'Maker_INFINITI', 'Maker_JAGUAR', 'Maker_JEEP', 'Maker_KIA',
       'Maker_LEXUS', 'Maker_MAZDA', 'Maker_MERCEDES-BENZ', 'Maker_MINI',
       'Maker_MITSUBISHI', 'Maker_NISSAN', 'Maker_OPEL', 'Maker_PEUGEOT',
       'Maker_RENAULT', 'Maker_SEAT', 'Maker_SKODA', 'Maker_SMART',
       'Maker_SSANGYONG', 'Maker_SUBARU', 'Maker_SUZUKI', 'Maker_TOYOTA',
       'Maker_VOLKSWAGEN', 'Maker_VOLVO', 'Fuel_Diesel',
       'Fuel_Gasoline','Maxvel_class_slow','Maxvel_class_avg','Maxvel_class_fast',
           'HP_class_low', 'HP_class_mid', 'HP_class_high','Year_Class_6_9anios', 
           'Year_Class_3_6anios', 'Year_Class_less_3anios']]


In [59]:

y  =db_ml['Price_euro']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)       
xg_reg = xgboost.XGBRegressor()
model = xg_reg.fit(X_train,y_train)
model.score(X_test, y_test)
          

0.44704841722651356

In [62]:
# Max vel bins, kilometers bins, years bins
X = db_ml[['HP', 'Kilometers_Class_High',
       'Kilometers_Class_Low', 'Kilometers_Class_Mid', 'Motor_Automatic',
       'Motor_Manual', 'Segment_SegmentA', 'Segment_SegmentB',
       'Segment_SegmentC', 'Segment_SegmentD', 'Segment_SegmentE',
       'Maker_ABARTH', 'Maker_ALFA', 'Maker_AUDI', 'Maker_BMW',
       'Maker_CHEVROLET', 'Maker_CITROEN', 'Maker_DACIA', 'Maker_DS',
       'Maker_FIAT', 'Maker_FORD', 'Maker_HONDA', 'Maker_HYUNDAI',
       'Maker_INFINITI', 'Maker_JAGUAR', 'Maker_JEEP', 'Maker_KIA',
       'Maker_LEXUS', 'Maker_MAZDA', 'Maker_MERCEDES-BENZ', 'Maker_MINI',
       'Maker_MITSUBISHI', 'Maker_NISSAN', 'Maker_OPEL', 'Maker_PEUGEOT',
       'Maker_RENAULT', 'Maker_SEAT', 'Maker_SKODA', 'Maker_SMART',
       'Maker_SSANGYONG', 'Maker_SUBARU', 'Maker_SUZUKI', 'Maker_TOYOTA',
       'Maker_VOLKSWAGEN', 'Maker_VOLVO', 'Fuel_Diesel',
       'Fuel_Gasoline','Maxvel_class_slow','Maxvel_class_avg','Maxvel_class_fast','Year_Class_6_9anios', 
           'Year_Class_3_6anios', 'Year_Class_less_3anios']]

In [63]:
y  =db_ml['Price_euro']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)       
xg_reg = xgboost.XGBRegressor()
model = xg_reg.fit(X_train,y_train)
model.score(X_test, y_test)

0.4953695787942092

In [64]:
X = db_ml[['HP', 'Kilometers', 'Motor_Automatic',
       'Motor_Manual', 'Segment_SegmentA', 'Segment_SegmentB',
       'Segment_SegmentC', 'Segment_SegmentD', 'Segment_SegmentE',
       'Maker_ABARTH', 'Maker_ALFA', 'Maker_AUDI', 'Maker_BMW',
       'Maker_CHEVROLET', 'Maker_CITROEN', 'Maker_DACIA', 'Maker_DS',
       'Maker_FIAT', 'Maker_FORD', 'Maker_HONDA', 'Maker_HYUNDAI',
       'Maker_INFINITI', 'Maker_JAGUAR', 'Maker_JEEP', 'Maker_KIA',
       'Maker_LEXUS', 'Maker_MAZDA', 'Maker_MERCEDES-BENZ', 'Maker_MINI',
       'Maker_MITSUBISHI', 'Maker_NISSAN', 'Maker_OPEL', 'Maker_PEUGEOT',
       'Maker_RENAULT', 'Maker_SEAT', 'Maker_SKODA', 'Maker_SMART',
       'Maker_SSANGYONG', 'Maker_SUBARU', 'Maker_SUZUKI', 'Maker_TOYOTA',
       'Maker_VOLKSWAGEN', 'Maker_VOLVO', 'Fuel_Diesel',
       'Fuel_Gasoline','Maxvel_class_slow','Maxvel_class_avg','Maxvel_class_fast','Year_Class_6_9anios', 
           'Year_Class_3_6anios', 'Year_Class_less_3anios']]

In [65]:
y  =db_ml['Price_euro']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)       
xg_reg = xgboost.XGBRegressor()
model = xg_reg.fit(X_train,y_train)
model.score(X_test, y_test)

0.5632508542401875

In [68]:
#With out bins
X = db_ml[['Year','HP', 'Kilometers', 'Motor_Automatic',
       'Motor_Manual', 'Segment_SegmentA', 'Segment_SegmentB',
       'Segment_SegmentC', 'Segment_SegmentD', 'Segment_SegmentE',
       'Maker_ABARTH', 'Maker_ALFA', 'Maker_AUDI', 'Maker_BMW',
       'Maker_CHEVROLET', 'Maker_CITROEN', 'Maker_DACIA', 'Maker_DS',
       'Maker_FIAT', 'Maker_FORD', 'Maker_HONDA', 'Maker_HYUNDAI',
       'Maker_INFINITI', 'Maker_JAGUAR', 'Maker_JEEP', 'Maker_KIA',
       'Maker_LEXUS', 'Maker_MAZDA', 'Maker_MERCEDES-BENZ', 'Maker_MINI',
       'Maker_MITSUBISHI', 'Maker_NISSAN', 'Maker_OPEL', 'Maker_PEUGEOT',
       'Maker_RENAULT', 'Maker_SEAT', 'Maker_SKODA', 'Maker_SMART',
       'Maker_SSANGYONG', 'Maker_SUBARU', 'Maker_SUZUKI', 'Maker_TOYOTA',
       'Maker_VOLKSWAGEN', 'Maker_VOLVO', 'Fuel_Diesel',
       'Fuel_Gasoline','Maxvel']]

In [69]:
y  =db_ml['Price_euro']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)       
xg_reg = xgboost.XGBRegressor()
model = xg_reg.fit(X_train,y_train)
model.score(X_test, y_test)

0.6720506628402307

In [70]:
y_pred = xg_reg.predict(X_test)

In [141]:
#Without Maxvel, the score is lowr
X = db_ml[['Year','HP', 'Kilometers', 'Motor_Automatic',
       'Motor_Manual', 'Segment_SegmentA', 'Segment_SegmentB',
       'Segment_SegmentC', 'Segment_SegmentD', 'Segment_SegmentE',
       'Maker_ABARTH', 'Maker_ALFA', 'Maker_AUDI', 'Maker_BMW',
       'Maker_CHEVROLET', 'Maker_CITROEN', 'Maker_DACIA', 'Maker_DS',
       'Maker_FIAT', 'Maker_FORD', 'Maker_HONDA', 'Maker_HYUNDAI',
       'Maker_INFINITI', 'Maker_JAGUAR', 'Maker_JEEP', 'Maker_KIA',
       'Maker_LEXUS', 'Maker_MAZDA', 'Maker_MERCEDES-BENZ', 'Maker_MINI',
       'Maker_MITSUBISHI', 'Maker_NISSAN', 'Maker_OPEL', 'Maker_PEUGEOT',
       'Maker_RENAULT', 'Maker_SEAT', 'Maker_SKODA', 'Maker_SMART',
       'Maker_SSANGYONG', 'Maker_SUBARU', 'Maker_SUZUKI', 'Maker_TOYOTA',
       'Maker_VOLKSWAGEN', 'Maker_VOLVO', 'Fuel_Diesel',
       'Fuel_Gasoline','Cylindernumber']]

In [142]:
len(X.columns)

47

In [143]:
#TEST with out Maxvel
y  =db_ml['Price_euro']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)       
xg_reg = xgboost.XGBRegressor()
model = xg_reg.fit(X_train,y_train)
model.score(X_test, y_test)

0.6603948757601816

In [75]:
#Using cross valdation
from sklearn.model_selection import cross_validate

model = xgboost.XGBRegressor()
results = cross_validate(model,X, y, cv = 10)
print(results['test_score'].mean())
results['test_score']

0.658191490201957


array([0.68988952, 0.66261545, 0.66623522, 0.63243383, 0.60775419,
       0.66576994, 0.69706611, 0.65203901, 0.66626472, 0.64184691])

In [78]:
np.mean(results['test_score'])

0.658191490201957

# MAIN ML MODEL:

In [144]:
#This is the best comination for features

X = db_ml[['Year','HP', 'Kilometers', 'Motor_Automatic',
       'Motor_Manual', 'Segment_SegmentA', 'Segment_SegmentB',
       'Segment_SegmentC', 'Segment_SegmentD', 'Segment_SegmentE',
       'Maker_ABARTH', 'Maker_ALFA', 'Maker_AUDI', 'Maker_BMW',
       'Maker_CHEVROLET', 'Maker_CITROEN', 'Maker_DACIA', 'Maker_DS',
       'Maker_FIAT', 'Maker_FORD', 'Maker_HONDA', 'Maker_HYUNDAI',
       'Maker_INFINITI', 'Maker_JAGUAR', 'Maker_JEEP', 'Maker_KIA',
       'Maker_LEXUS', 'Maker_MAZDA', 'Maker_MERCEDES-BENZ', 'Maker_MINI',
       'Maker_MITSUBISHI', 'Maker_NISSAN', 'Maker_OPEL', 'Maker_PEUGEOT',
       'Maker_RENAULT', 'Maker_SEAT', 'Maker_SKODA', 'Maker_SMART',
       'Maker_SSANGYONG', 'Maker_SUBARU', 'Maker_SUZUKI', 'Maker_TOYOTA',
       'Maker_VOLKSWAGEN', 'Maker_VOLVO', 'Fuel_Diesel',
       'Fuel_Gasoline','Maxvel','Cylindernumber']]

In [145]:
len(X.columns)

48

In [146]:
y  =db_ml['Price_euro']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)       
xg_reg = xgboost.XGBRegressor(max_depth=3, n_estimators=200, random_state=1)
model = xg_reg.fit(X_train,y_train)
model.score(X_test, y_test)

0.7015945496875393

In [83]:
#pickle model
pickle.dump(xg_reg, open('model.p', 'wb'))

In [215]:
#pickle xtest and ytest
X_test.to_pickle('xtest.p')
y_test.to_pickle('ytest.p')

In [95]:
from sklearn.metrics import mean_squared_error
y_pred = xg_reg.predict(X_test)
display(mean_squared_error(y_test, y_pred))
display(np.sqrt(mean_squared_error(y_test, y_pred)))

1278506.0566663672

1130.7104212248012

In [96]:
from sklearn.metrics import mean_absolute_error
scoremae = mean_absolute_error(y_test, y_pred)
scoremae

884.4507382903661

In [97]:
from sklearn.model_selection import cross_validate

model = xgboost.XGBRegressor()
results = cross_validate(model,X, y, cv = 10)
print(results)

{'fit_time': array([0.58263898, 0.54166818, 0.57664299, 0.57064915, 0.55365968,
       0.67558432, 0.60262823, 0.59463429, 0.57864285, 0.55165958]), 'score_time': array([0.01498985, 0.01299262, 0.01399231, 0.01499081, 0.01499152,
       0.01499176, 0.01499104, 0.01399255, 0.01599026, 0.01499081]), 'test_score': array([0.71299162, 0.66986933, 0.71304044, 0.67957172, 0.64398393,
       0.69417971, 0.71855182, 0.69727192, 0.69226547, 0.67913023])}


In [98]:
print(results['test_score'].mean())
results['test_score']

0.6900856170904769


array([0.71299162, 0.66986933, 0.71304044, 0.67957172, 0.64398393,
       0.69417971, 0.71855182, 0.69727192, 0.69226547, 0.67913023])

In [99]:
def average(list): 
    result = sum(list) / len(list) 
    return result 

In [100]:
average(results['test_score'])

0.6900856170904769

Hyperparameters tunning

In [52]:
from sklearn.model_selection import GridSearchCV

X = db_ml[['Year','HP', 'Kilometers', 'Motor_Automatic',
       'Motor_Manual', 'Segment_SegmentA', 'Segment_SegmentB',
       'Segment_SegmentC', 'Segment_SegmentD', 'Segment_SegmentE',
       'Maker_ABARTH', 'Maker_ALFA', 'Maker_AUDI', 'Maker_BMW',
       'Maker_CHEVROLET', 'Maker_CITROEN', 'Maker_DACIA', 'Maker_DS',
       'Maker_FIAT', 'Maker_FORD', 'Maker_HONDA', 'Maker_HYUNDAI',
       'Maker_INFINITI', 'Maker_JAGUAR', 'Maker_JEEP', 'Maker_KIA',
       'Maker_LEXUS', 'Maker_MAZDA', 'Maker_MERCEDES-BENZ', 'Maker_MINI',
       'Maker_MITSUBISHI', 'Maker_NISSAN', 'Maker_OPEL', 'Maker_PEUGEOT',
       'Maker_RENAULT', 'Maker_SEAT', 'Maker_SKODA', 'Maker_SMART',
       'Maker_SSANGYONG', 'Maker_SUBARU', 'Maker_SUZUKI', 'Maker_TOYOTA',
       'Maker_VOLKSWAGEN', 'Maker_VOLVO', 'Fuel_Diesel',
       'Fuel_Gasoline','Maxvel','Cylindernumber']]

y  =db_ml['Price_euro']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)  

hyperparameter_grid = {
    'n_estimators': [100, 400, 800],
    'max_depth': [3, 6, 9],
    'learning_rate': [0.05, 0.1, 0.20],
    'min_child_weight': [1, 10, 100]
    }

# estimator -> model to optimize 
xgboost =xgboost.XGBRegressor()
# cv = 3 -> number of cross validation folds <------ CV is REALLY important in grid search. Why?
grid_search = GridSearchCV(estimator = xgboost, param_grid = hyperparameter_grid, cv = 3)
# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# and the winner is...
print(grid_search.best_params_)

# in grid search you are more likely to get really good results in your training set, even with CV
print(grid_search.best_score_)

{'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 800}
0.6814745625237414


In [103]:
from sklearn.model_selection import GridSearchCV

X = db_ml[['Year','HP', 'Kilometers', 'Motor_Automatic',
       'Motor_Manual', 'Segment_SegmentA', 'Segment_SegmentB',
       'Segment_SegmentC', 'Segment_SegmentD', 'Segment_SegmentE',
       'Maker_ABARTH', 'Maker_ALFA', 'Maker_AUDI', 'Maker_BMW',
       'Maker_CHEVROLET', 'Maker_CITROEN', 'Maker_DACIA', 'Maker_DS',
       'Maker_FIAT', 'Maker_FORD', 'Maker_HONDA', 'Maker_HYUNDAI',
       'Maker_INFINITI', 'Maker_JAGUAR', 'Maker_JEEP', 'Maker_KIA',
       'Maker_LEXUS', 'Maker_MAZDA', 'Maker_MERCEDES-BENZ', 'Maker_MINI',
       'Maker_MITSUBISHI', 'Maker_NISSAN', 'Maker_OPEL', 'Maker_PEUGEOT',
       'Maker_RENAULT', 'Maker_SEAT', 'Maker_SKODA', 'Maker_SMART',
       'Maker_SSANGYONG', 'Maker_SUBARU', 'Maker_SUZUKI', 'Maker_TOYOTA',
       'Maker_VOLKSWAGEN', 'Maker_VOLVO', 'Fuel_Diesel',
       'Fuel_Gasoline','Maxvel','Cylindernumber']]

y  =db_ml['Price_euro']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)  

hyperparameter_grid = {
    'n_estimators': [100,150,200,250,300,400, 800],
    'max_depth': [1,3,6, 9],
    'learning_rate': [0.05, 0.1, 0.20],
    'min_child_weight': [1, 10, 100]
    }

# estimator -> model to optimize 
xgboost = xgboost.XGBRegressor()
# cv = 3 -> number of cross validation folds <------ CV is REALLY important in grid search. Why?
grid_search = GridSearchCV(estimator = xgboost, param_grid = hyperparameter_grid, cv = 3)
# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# and the winner is...
print(grid_search.best_params_)

# in grid search you are more likely to get really good results in your training set, even with CV
print(grid_search.best_score_)

{'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 800}
0.6764357757065139


In [56]:
from sklearn.model_selection import RandomizedSearchCV

X = db_ml[['Year','HP', 'Kilometers', 'Motor_Automatic',
       'Motor_Manual', 'Segment_SegmentA', 'Segment_SegmentB',
       'Segment_SegmentC', 'Segment_SegmentD', 'Segment_SegmentE',
       'Maker_ABARTH', 'Maker_ALFA', 'Maker_AUDI', 'Maker_BMW',
       'Maker_CHEVROLET', 'Maker_CITROEN', 'Maker_DACIA', 'Maker_DS',
       'Maker_FIAT', 'Maker_FORD', 'Maker_HONDA', 'Maker_HYUNDAI',
       'Maker_INFINITI', 'Maker_JAGUAR', 'Maker_JEEP', 'Maker_KIA',
       'Maker_LEXUS', 'Maker_MAZDA', 'Maker_MERCEDES-BENZ', 'Maker_MINI',
       'Maker_MITSUBISHI', 'Maker_NISSAN', 'Maker_OPEL', 'Maker_PEUGEOT',
       'Maker_RENAULT', 'Maker_SEAT', 'Maker_SKODA', 'Maker_SMART',
       'Maker_SSANGYONG', 'Maker_SUBARU', 'Maker_SUZUKI', 'Maker_TOYOTA',
       'Maker_VOLKSWAGEN', 'Maker_VOLVO', 'Fuel_Diesel',
       'Fuel_Gasoline','Maxvel','Cylindernumber']]

y  =db_ml['Price_euro']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)  

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)] #10
max_depth = [int(x) for x in np.linspace(1, 15, num = 1)]
learning_rate = [float(x) for x in np.random.uniform(low=0.5, high=.24, size=(50,))]
min_child_weight = [int(x) for x in np.linspace(1, 100, num = 5)]

random_grid = {
    'n_estimators':n_estimators,
    'max_depth':max_depth,
    'learning_rate': learning_rate,
    'min_child_weight': min_child_weight
    }

# estimator -> model to optimize 
xgboost =xgboost

random_search = RandomizedSearchCV(estimator = xgboost, param_distributions = random_grid, n_iter = 15, cv = 3, n_jobs = 15)

random_search.fit(X_train,y_train)

print(random_search.best_params_)
print(random_search.best_score_)
print(random_search.best_score_)

{'n_estimators': 1600, 'min_child_weight': 25, 'max_depth': 1, 'learning_rate': 0.3873031067160606}
0.62876765855179
0.62876765855179


After trying to find the best hyperparameter we didnt find a better option that we had.

In [104]:
db[['Year','Maker','HP', 'Kilometers', 'Motor','Segment','Fuel','Maxvel','Cylindernumber','Price_euro']].iloc[40:41]

,Year,Maker,HP,Kilometers,Motor,Segment,Fuel,Maxvel,Cylindernumber,Price_euro
40,2019,SUZUKI,88.7328,29444,Manual,SegmentB,Gasoline,180.0,4,13900


In [116]:
X = db_ml[['Year','HP', 'Kilometers', 'Motor_Automatic',
       'Motor_Manual', 'Segment_SegmentA', 'Segment_SegmentB',
       'Segment_SegmentC', 'Segment_SegmentD', 'Segment_SegmentE',
       'Maker_ABARTH', 'Maker_ALFA', 'Maker_AUDI', 'Maker_BMW',
       'Maker_CHEVROLET', 'Maker_CITROEN', 'Maker_DACIA', 'Maker_DS',
       'Maker_FIAT', 'Maker_FORD', 'Maker_HONDA', 'Maker_HYUNDAI',
       'Maker_INFINITI', 'Maker_JAGUAR', 'Maker_JEEP', 'Maker_KIA',
       'Maker_LEXUS', 'Maker_MAZDA', 'Maker_MERCEDES-BENZ', 'Maker_MINI',
       'Maker_MITSUBISHI', 'Maker_NISSAN', 'Maker_OPEL', 'Maker_PEUGEOT',
       'Maker_RENAULT', 'Maker_SEAT', 'Maker_SKODA', 'Maker_SMART',
       'Maker_SSANGYONG', 'Maker_SUBARU', 'Maker_SUZUKI', 'Maker_TOYOTA',
       'Maker_VOLKSWAGEN', 'Maker_VOLVO', 'Fuel_Diesel',
       'Fuel_Gasoline','Maxvel','Cylindernumber']]

In [117]:
test = [[2015,88, 29444,0,
                    1,0,1,
                    0,0,0,
                    0,0,0,0,
                    0,0,0,0,
                    0,0,0,0,
                    0,0,0,0,
                    0,0,0,0,
                    0,0,0,0,
                    0,0,0,0,
                    0,0,1,0,
                    0,0,0,
                    1,180,4]]
arr = np.array(test)
RESULT =model.predict(arr)
print('Price: ',round(RESULT[0]))
print('Price: ',round(RESULT[0]*.68),'-',round(RESULT[0]*1.22))


Price:  11905
Price:  8095 - 14524


In [118]:
db[['Year','Maker','HP', 'Kilometers', 'Motor','Segment','Fuel','Maxvel','Cylindernumber','Price_euro']].iloc[410:411]

,Year,Maker,HP,Kilometers,Motor,Segment,Fuel,Maxvel,Cylindernumber,Price_euro
410,2018,OPEL,73.944,153192,Manual,SegmentC,Diesel,164.0,4,8900


In [119]:
test = [[2018,74, 153192,0,
       1, 0, 0,
       1, 0, 0,
       0, 0, 0, 0,
       0, 0, 0, 0,
       0, 0, 0, 0,
       0, 0, 0, 0,
       0, 0, 0,0,
       0, 0, 1,0,
       0, 0, 0, 0,
       0, 0, 0, 0,
       0, 0, 1,
       0,164.0,4]]
len(test[0])

48

In [120]:
arr = np.array(test)
RESULT = model.predict(arr)
print('Price: ',round(RESULT[0]))
print('Price: ',round(RESULT[0]*.68),'-',round(RESULT[0]*1.22)) 
print('Price: ',round(RESULT[0]-scoremae),'-',round(RESULT[0]+scoremae))           

Price:  9506
Price:  6464 - 11597
Price:  8621 - 10390


In [ ]:
X = db_ml[['Year','HP', 'Kilometers', 'Motor_Automatic',
       'Motor_Manual', 'Segment_SegmentA', 'Segment_SegmentB',
       'Segment_SegmentC', 'Segment_SegmentD', 'Segment_SegmentE',
       'Maker_ABARTH', 'Maker_ALFA', 'Maker_AUDI', 'Maker_BMW',
       'Maker_CHEVROLET', 'Maker_CITROEN', 'Maker_DACIA', 'Maker_DS',
       'Maker_FIAT', 'Maker_FORD', 'Maker_HONDA', 'Maker_HYUNDAI',
       'Maker_INFINITI', 'Maker_JAGUAR', 'Maker_JEEP', 'Maker_KIA',
       'Maker_LEXUS', 'Maker_MAZDA', 'Maker_MERCEDES-BENZ', 'Maker_MINI',
       'Maker_MITSUBISHI', 'Maker_NISSAN', 'Maker_OPEL', 'Maker_PEUGEOT',
       'Maker_RENAULT', 'Maker_SEAT', 'Maker_SKODA', 'Maker_SMART',
       'Maker_SSANGYONG', 'Maker_SUBARU', 'Maker_SUZUKI', 'Maker_TOYOTA',
       'Maker_VOLKSWAGEN', 'Maker_VOLVO', 'Fuel_Diesel',
       'Fuel_Gasoline','Maxvel','Cylindernumber']]

In [ ]:
len(X.columns)

In [ ]:
#falta abrir pickle de X_train y ajustar index a clumpnas de este dataset
def predict_price(Year,HP,Kilometers,Automatic_Manual,Segmet_ABCDEF,Maker,Fuel,Max_vel,Cylindernumber):
    templist = [0,0, 0,0,
                    0,0,0,
                    0,0,0,
                    0,0,0,0,
                    0,0,0,0,
                    0,0,0,0,
                    0,0,0,0,
                    0,0,0,0,
                    0,0,0,0,
                    0,0,0,0,
                    0,0,0,0,
                    0,0,0,
                    0,0,0]
        
    year = int(Year)
    templist[0] = year
    
    HP = int(HP) 
    templist[1] = HP
    
    Kilometers = int(Kilometers)
    templist[2] = Kilometers
    
    motor= 'Motor_'+str(Automatic_Manual).capitalize()
    motorindx =X.columns.tolist().index(motor)
    templist[motorindx] = 1
    
    segment = 'Segment_Segment'+str(Segmet_ABCDEF).capitalize()
    segmentindx =X.columns.tolist().index(segment)
    templist[segmentindx] = 1
    
    maker = 'Maker_'+str(Maker).upper()
    makertindx =X.columns.tolist().index(maker)
    templist[makertindx] = 1
    
    fuel = 'Fuel_'+str(Fuel).capitalize()
    fuelindx =X.columns.tolist().index(fuel)
    templist[fuelindx] = 1
    
    maxvel = int(Max_vel)
    templist[46] = maxvel
    
    
    Cylindernumber = int(Cylindernumber)
    templist[47] = Cylindernumber
    
     
    #year,HP,Kilometers,motor,segment,maker,fuel,maxvel,Cylindernumber
    
    return print(templist)

In [ ]:
predict_price(2015,90,73375,'automatic','b','nissan','diEsel',180,4)

In [ ]:
db[['Year','Maker','HP', 'Kilometers', 'Motor','Segment','Fuel','Maxvel','Cylindernumber','Price_euro']].iloc[410:411]

In [ ]:
example = predict_price(2018,73.944,153192,'Manual','c','opel','diEsel',164,4)

In [ ]:
len([2018, 73, 153192, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 164, 4])

In [ ]:
model.predict(np.array([[2018, 73, 153192, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 164, 4]]))

In [ ]:
test = [[2018,74, 153192,0,
                    1,0,0,
                    1,0,0,
                    0,0,0,0,
                    0,0,0,0,
                    0,0,0,0,
                    0,0,0,0,
                    0,0,0,0,
                    0,0,0,0,
                    0,0,0,0,
                    0,0,1,0,
                    0,0,1,
                    0,164,4]]

In [ ]:
X.columns.tolist().index('Motor_Automatic')

In [ ]:
X.columns.tolist().index('Maker_OPEL')

In [ ]:
Fuel = 'diesel'

In [ ]:
fuel = 'Fuel_'+str(Fuel).capitalize()
fuelindx =db_ml.columns.tolist().index(fuel)
print(fuelindx)
#templist[fuelindx] = 1

In [ ]:
['0', '0', '0', '0', '0',
       '0', '0', '0',
       '0', '0', '0',
       '0', '0', '0',
       'Maker_ABARTH', 'Maker_ALFA', 'Maker_AUDI', 'Maker_BMW',
       'Maker_CHEVROLET', 'Maker_CITROEN', 'Maker_DACIA', 'Maker_DS',
       'Maker_FIAT', 'Maker_FORD', 'Maker_HONDA', 'Maker_HYUNDAI',
       'Maker_INFINITI', 'Maker_JAGUAR', 'Maker_JEEP', 'Maker_KIA',
       'Maker_LEXUS', 'Maker_MAZDA', 'Maker_MERCEDES-BENZ', 'Maker_MINI',
       'Maker_MITSUBISHI', 'Maker_NISSAN', 'Maker_OPEL', 'Maker_PEUGEOT',
       'Maker_RENAULT', 'Maker_SEAT', 'Maker_SKODA', 'Maker_SMART',
       'Maker_SSANGYONG', 'Maker_SUBARU', 'Maker_SUZUKI', 'Maker_TOYOTA',
       'Maker_VOLKSWAGEN', 'Maker_VOLVO', 'Fuel_Diesel', 'Fuel_Gasoline',
       'Price_euro', 'Cylindernumber', 'Maxvel_class_slow', 'Maxvel_class_avg',
       'Maxvel_class_fast', 'HP_class_low', 'HP_class_mid', 'HP_class_high',
       'Year_Class_6_9anios', 'Year_Class_3_6anios', 'Year_Class_less_3anios']

# MAIN Function for ML 

In [125]:
def predict_price(Year,HP,Kilometers,Automatic_Manual,Segmet_ABCDEF,Maker,Fuel,Max_vel,Cylindernumber):
    X_test = pd.read_pickle('xtest.p')
    y_test = pd.read_pickle('ytest.p')
    templist = [0,0, 0,0,
                    0,0,0,
                    0,0,0,
                    0,0,0,0,
                    0,0,0,0,
                    0,0,0,0,
                    0,0,0,0,
                    0,0,0,0,
                    0,0,0,0,
                    0,0,0,0,
                    0,0,0,0,
                    0,0,0,
                    0,0,0]
        
    year = int(Year)
    templist[0] = year
    
    HP = int(HP) 
    templist[1] = HP
    
    Kilometers = int(Kilometers)
    templist[2] = Kilometers
    
    motor= 'Motor_'+str(Automatic_Manual).capitalize()
    motorindx =X_test.columns.tolist().index(motor)
    templist[motorindx] = 1
    
    segment = 'Segment_Segment'+str(Segmet_ABCDEF).capitalize()
    segmentindx =X_test.columns.tolist().index(segment)
    templist[segmentindx] = 1
    
    maker = 'Maker_'+str(Maker).upper()
    makertindx =X_test.columns.tolist().index(maker)
    templist[makertindx] = 1
    
    fuel = 'Fuel_'+str(Fuel).capitalize()
    fuelindx =X_test.columns.tolist().index(fuel)
    templist[fuelindx] = 1
    
    maxvel = int(Max_vel)
    templist[46] = maxvel
    
    
    Cylindernumber = int(Cylindernumber)
    templist[47] = Cylindernumber
    
    xgb_model = pickle.load(open('model.p','rb'))
    score = xgb_model.score(X_test,y_test)
    score = round(score*100,3)
    
    y_pred = xgb_model.predict(X_test)
    scoremae=mean_absolute_error(y_test, y_pred)
    #year,HP,Kilometers,motor,segment,maker,fuel,maxvel,Cylindernumber
    
    result = xgb_model.predict(np.array([templist]))
    
    result_range1 = round(result[0]-scoremae)
    result_range2 = round(result[0]+scoremae)
    
    return print('Prediction: ',result[0],'Score: ', score
                 ,'Price range: ',result_range1,'-',result_range2,"MAE: ",scoremae)

# ML prediction model trial

In [126]:
predict_price(2018,73.944,153192,'manual','c','opel','diEsel',164.0,4)

Prediction:  9331.43 Score:  80.595 Price range:  8604 - 10059 MAE:  727.2222186770712


In [127]:
print('Price: ',(9485.254-scoremae),'-',(9485.254+scoremae))      

Price:  8600.803261709634 - 10369.704738290367


In [128]:
db[['Year','Maker','HP', 'Kilometers', 'Motor','Segment','Fuel','Maxvel','Cylindernumber','Price_euro']].iloc[410:411]

,Year,Maker,HP,Kilometers,Motor,Segment,Fuel,Maxvel,Cylindernumber,Price_euro
410,2018,OPEL,73.944,153192,Manual,SegmentC,Diesel,164.0,4,8900


In [129]:
db[['Year','Maker','HP', 'Kilometers', 'Motor','Segment','Fuel','Maxvel','Cylindernumber','Price_euro']].iloc[1410:1411]

,Year,Maker,HP,Kilometers,Motor,Segment,Fuel,Maxvel,Cylindernumber,Price_euro
1410,2014,NISSAN,108.4512,130000,Manual,SegmentC,Diesel,175.0,4,12500


In [130]:
predict_price(2014,108.4512,130000,'manual','c','nissan','diEsel',175,4)

Prediction:  12901.798 Score:  80.595 Price range:  12175 - 13629 MAE:  727.2222186770712


In [131]:
predict_price(2015,147.888,236000,'manual','D','BMW','diEsel',205,4)

Prediction:  13067.943 Score:  80.595 Price range:  12341 - 13795 MAE:  727.2222186770712


In [132]:
print('Price: ',(11532.986-scoremae),'-',(11532.986+scoremae))    

Price:  10648.535261709634 - 12417.436738290367


In [133]:
print('Price: ',(11532.986-scoremae),'-',(11532.986+scoremae))       

Price:  10648.535261709634 - 12417.436738290367


Analizing the proce of a car in the test dataset

In [134]:
y_test

1240    15000
3014    14300
3269    13500
1036    12450
2465    10350
        ...  
1742    11200
1965     8500
591      8290
1160    12999
2523    13490
Name: Price_euro, Length: 857, dtype: int64

In [135]:
db[['Year','Maker','HP', 'Kilometers', 'Motor','Segment','Fuel','Maxvel','Cylindernumber','Price_euro']].iloc[1036:1037]

,Year,Maker,HP,Kilometers,Motor,Segment,Fuel,Maxvel,Cylindernumber,Price_euro
1036,2019,OPEL,88.7328,32621,Manual,SegmentC,Gasoline,175.0,4,12450


In [136]:
predict_price(2019,88.7328,32621,'manual','c','OPEL','Gasoline',175,4)

Prediction:  13124.279 Score:  80.595 Price range:  12397 - 13852 MAE:  727.2222186770712


In [139]:
predict_price(2016,138,34000,'manual','c','OPEL','Gasoline',215,4)

Prediction:  15052.847 Score:  80.595 Price range:  14326 - 15780 MAE:  727.2222186770712


In [140]:
predict_price(2016,99,135000,'manual','b','PEUGEOT','diesel',230,4)

Prediction:  10907.744 Score:  80.595 Price range:  10181 - 11635 MAE:  727.2222186770712


https://www.coches.net/peugeot-208-5p-allure-16-bluehdi-73kw-100cv-diesel-2016-en-madrid-52608921-covo.aspx

# Unique values for Flask app select pickle creation

In [97]:
maker_vals = db['Maker'].unique()
pd.DataFrame(maker_vals, columns =['Maker']).to_pickle('maker_vals.p')

In [65]:
def roundd(num):
    return round(num,0)

In [67]:
roundd(88.7328)

89.0

In [91]:
year_vals = db['Year'].round().unique()
pd.DataFrame(year_vals, columns =['Year']).to_pickle('year_vals.p')

In [79]:
db['HP'].round().unique()

array([ 89.,  74., 134.,  81., 104., 118., 114., 128.,  99., 123., 108.,
       148.,  68.,  94., 112.,  72., 138., 113., 130.,  76.,  65.,  66.,
        79.,  84.,  69., 107., 109.,  67.,  71.,  77., 110., 158.,  70.,
       120., 133., 163., 168.,  80.,  86.,  98.,  59.,  82.,  83., 187.,
        91., 161., 101., 139., 132., 145.,  93., 141., 147., 197., 126.,
       175., 201., 115., 119.,  64., 116., 164., 122., 181., 177., 192.,
       153.,  85., 129.,  97., 154., 140., 178.])

In [101]:
segment_vals = db['Segment'].unique()
pd.DataFrame(segment_vals, columns =['Segment']).to_pickle('segment_vals.p')
segment_vals

array(['SegmentB', 'SegmentC', 'SegmentE', 'SegmentD', 'SegmentA'],
      dtype=object)

In [94]:
fuel_vals = db['Fuel'].unique()
pd.DataFrame(fuel_vals, columns =['Fuel']).to_pickle('fuel_vals.p')

In [82]:
db['Maxvel'].unique()

array([175.        , 178.        , 173.        , 168.        ,
       210.        , 174.        , 185.        , 181.        ,
       195.        , 171.        , 184.        , 198.        ,
       187.        , 215.        , 160.        , 180.        ,
       172.        , 203.        , 197.        , 177.        ,
       192.        , 190.        , 211.        , 200.        ,
       181.50646228, 179.        , 201.        , 213.        ,
       165.        , 155.        , 158.        , 161.        ,
       170.        , 163.        , 182.        , 167.        ,
       186.        , 188.        , 150.        , 164.        ,
       205.        , 209.        , 193.        , 176.        ,
       169.        , 183.        , 208.        , 194.        ,
       230.        , 162.        , 152.        , 204.        ,
       216.        , 207.        , 217.        , 199.        ,
       196.        , 220.        , 191.        , 189.        ,
       156.        , 218.        , 157.        , 159.  

In [102]:
cylindernumber_vals = db['Cylindernumber'].unique()
pd.DataFrame(cylindernumber_vals, columns =['Cylindernumber']).to_pickle('cylindernumber_vals.p')
cylindernumber_vals

array([3, 4, 2, 5], dtype=int64)

In [100]:
maker_vals = pd.read_pickle('maker_vals.p')
list(maker_vals['Maker'])

['NISSAN',
 'RENAULT',
 'KIA',
 'OPEL',
 'CITROEN',
 'MAZDA',
 'FIAT',
 'BMW',
 'VOLKSWAGEN',
 'DACIA',
 'FORD',
 'SKODA',
 'SUZUKI',
 'SEAT',
 'DS',
 'JEEP',
 'PEUGEOT',
 'TOYOTA',
 'HYUNDAI',
 'VOLVO',
 'SMART',
 'MERCEDES-BENZ',
 'ALFA',
 'MINI',
 'ABARTH',
 'MITSUBISHI',
 'INFINITI',
 'HONDA',
 'AUDI',
 'SSANGYONG',
 'CHEVROLET',
 'JAGUAR',
 'SUBARU',
 'LEXUS']